In [ ]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2026-01-31 01:31:47--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2026-01-31 01:31:47 (8.46 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2026-01-31 01:31:47--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
train_x = pd.read_csv(train_file_path, sep='\t', header=None)

train_y = train_x.pop(0)
train_y = train_y.map(lambda x: 0 if x == 'ham' else 1)

valid_x = pd.read_csv(test_file_path, sep='\t', header=None)

valid_y = valid_x.pop(0)
valid_y = valid_y.map(lambda x: 0 if x == 'ham' else 1)

In [ ]:
VOCAB_SIZE = 10000
MAX_LENGTH = 150

vec_layer = keras.layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_LENGTH)
vec_layer.adapt(train_x.values)

def make_ds(dfx,dfy):
  return tf.data.Dataset.from_tensor_slices((dfx,dfy)).shuffle(5000).batch(32).prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(train_x, train_y)
valid_ds = make_ds(valid_x, valid_y)

In [ ]:
model = keras.Sequential([
vec_layer,
keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=64, mask_zero=True),
keras.layers.LSTM(64, use_cudnn=False),
keras.layers.Dense(32, activation='relu'),
keras.layers.Dropout(0.2),
keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics=['acc'])

In [ ]:
history = model.fit(train_ds, validation_data=valid_ds, verbose=1, epochs=3)

Epoch 1/3
131/131 ━━━━━━━━━━━━━━━━━━━━ 51s 369ms/step - acc: 0.8868 - loss: 0.4091 - val_acc: 0.9842 - val_loss: 0.0529
Epoch 2/3
131/131 ━━━━━━━━━━━━━━━━━━━━ 80s 359ms/step - acc: 0.9907 - loss: 0.0420 - val_acc: 0.9878 - val_loss: 0.0409
Epoch 3/3
131/131 ━━━━━━━━━━━━━━━━━━━━ 84s 373ms/step - acc: 0.9965 - loss: 0.0145 - val_acc: 0.9892 - val_loss: 0.0427


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  input_tensor = tf.constant([pred_text])

  prob = model.predict(input_tensor, verbose=0)[0][0]

  label = 'spam' if prob > 0.5 else 'ham'
  return [float(prob), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[5.304486694512889e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
